In [2]:
import pandas as pd

df = pd.read_csv("crt_states.csv", usecols=['state', 'bill_number', 'title', 'bill_text'])
df.head()

,state,bill_number,title,bill_text
0,UT,HR0901,House Resolution on Critical Race Theory in Pu...,Enrolled Copy H.R. 901\n\n1 HOUSE RESOLUTION O...
1,UT,SR0901,Senate Resolution on Critical Race Theory in P...,Enrolled Copy S.R. 901\n\n1 SENATE RESOLUTION ...
2,IL,HR0365,CRITICAL RACE THEORY GRANTS,Illinois-2021-HR0365-Introduced\n\n\n\n\t \n\n...
3,IL,HB4066,CRITICAL RACE THEORY ACADEMY,Illinois-2021-HB4066-Introduced\n\n\n\n\t \n\n...
4,OK,HR1038,Resolution; discouraging schools from mandatin...,Resolution \n\nENROLLED HOUSE \n\nRESOLUTION N...


In [5]:
df['bill_id'] = df.state + "-" + df.bill_number
df = df.set_index('bill_id')
df.head()

,state,bill_number,title,bill_text
bill_id,,,,
UT-HR0901,UT,HR0901,House Resolution on Critical Race Theory in Pu...,Enrolled Copy H.R. 901\n\n1 HOUSE RESOLUTION O...
UT-SR0901,UT,SR0901,Senate Resolution on Critical Race Theory in P...,Enrolled Copy S.R. 901\n\n1 SENATE RESOLUTION ...
IL-HR0365,IL,HR0365,CRITICAL RACE THEORY GRANTS,Illinois-2021-HR0365-Introduced\n\n\n\n\t \n\n...
IL-HB4066,IL,HB4066,CRITICAL RACE THEORY ACADEMY,Illinois-2021-HB4066-Introduced\n\n\n\n\t \n\n...
OK-HR1038,OK,HR1038,Resolution; discouraging schools from mandatin...,Resolution \n\nENROLLED HOUSE \n\nRESOLUTION N...


In [6]:
df.shape

(32, 4)

### STEP ONE: Vectorize!

* We used Tfidf because it makes common phrases less important, and bills are full of common stock phrases like "I present to the speaker"
* We used ngram_range of (1,3) because... it seemed right? We can always change it later.

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(3, 6))

matrix = vectorizer.fit_transform(df.bill_text)
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names(),
                        index=df.index)

words_df.head()

,000 000 from,000 000 from the,000 000 from the general,000 000 from the general fund,000 000 in,000 000 in fiscal,000 000 in fiscal year,000 000 in fiscal year 2021,000 000 is,000 000 is for,...,zoning commission or zoning board,zoning commission or zoning board of,zoning commission planning,zoning commission planning and,zoning commission planning and zoning,zoning commission planning and zoning commission,zoning districts when,zoning districts when an,zoning districts when an application,zoning districts when an application petition
bill_id,,,,,,,,,,,,,,,,,,,,,
UT-HR0901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UT-SR0901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IL-HR0365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IL-HB4066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OK-HR1038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Let's narrow our pool of possible text reusers

* We'll use a similarity matrix!

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the similarities using the word counts
similarities = cosine_similarity(matrix)

# Make a fancy colored dataframe about it
similarity = pd.DataFrame(similarities, index=df.index, columns=df.index)

similarity.style.background_gradient(axis=None)

bill_id,UT-HR0901,UT-SR0901,IL-HR0365,IL-HB4066,OK-HR1038,LA-HSR3,WV-SB618,SC-H4325,AL-HB11,MO-HB952,MI-SB0460,AL-HB8,MS-HR87,MS-HC62,MS-SR56,HI-SCR66,MA-H689,HI-SR48,ID-H0375,ID-H0377,WA-SB5194,DC-PR24-0018,CA-AB1322,HI-HR142,HI-HCR168,CO-HB1250,WA-HB1477,CA-AB473,MN-SF165,MN-SF2015,MN-HF6,CT-HB06666
bill_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
UT-HR0901,1.000000,0.839337,0.011486,0.002986,0.011413,0.021928,0.020014,0.005316,0.003721,0.004916,0.009060,0.007892,0.023081,0.018142,0.014411,0.014205,0.001357,0.013598,0.003683,0.012157,0.001025,0.000000,0.001530,0.011829,0.011779,0.001380,0.001679,0.000471,0.000502,0.000502,0.000640,0.000460
UT-SR0901,0.839337,1.000000,0.008121,0.003006,0.004966,0.017115,0.020150,0.005352,0.003746,0.004950,0.009121,0.007946,0.010171,0.010113,0.021969,0.016955,0.000857,0.017034,0.002967,0.011510,0.001249,0.000000,0.001540,0.007561,0.007529,0.000410,0.000692,0.000474,0.000483,0.000483,0.000236,0.000264
IL-HR0365,0.011486,0.008121,1.000000,0.004469,0.004553,0.012002,0.002958,0.007223,0.000738,0.003488,0.001523,0.000972,0.011700,0.011280,0.008588,0.007858,0.001232,0.008404,0.001187,0.001314,0.001422,0.000229,0.000163,0.009121,0.008251,0.002035,0.000651,0.000705,0.001362,0.001362,0.000220,0.000215
IL-HB4066,0.002986,0.003006,0.004469,1.000000,0.001107,0.003534,0.002705,0.003972,0.000994,0.002835,0.005397,0.002649,0.001544,0.001535,0.001369,0.000601,0.010241,0.000604,0.001414,0.001529,0.004038,0.000142,0.001464,0.000642,0.000640,0.011549,0.004079,0.003974,0.003456,0.003456,0.001591,0.008151
OK-HR1038,0.011413,0.004966,0.004553,0.001107,1.000000,0.009895,0.052190,0.006388,0.005270,0.003582,0.007628,0.039869,0.053477,0.052227,0.041070,0.003575,0.001101,0.003028,0.007100,0.007662,0.003055,0.000951,0.000154,0.006771,0.006742,0.002874,0.001752,0.001999,0.000983,0.000983,0.000551,0.004891
LA-HSR3,0.021928,0.017115,0.012002,0.003534,0.009895,1.000000,0.007097,0.008211,0.001714,0.005561,0.002275,0.001451,0.008293,0.006873,0.003993,0.008713,0.003460,0.007933,0.002749,0.003219,0.002062,0.000000,0.002226,0.004445,0.004426,0.006024,0.002127,0.002091,0.001253,0.001253,0.001036,0.000693
WV-SB618,0.020014,0.020150,0.002958,0.002705,0.052190,0.007097,1.000000,0.017056,0.017843,0.005264,0.040673,0.299935,0.075814,0.075379,0.073232,0.000765,0.003023,0.000768,0.008024,0.012794,0.003311,0.000000,0.000424,0.000889,0.000886,0.008034,0.003352,0.004877,0.001389,0.001389,0.004085,0.009258
SC-H4325,0.005316,0.005352,0.007223,0.003972,0.006388,0.008211,0.017056,1.000000,0.143832,0.010521,0.173697,0.020303,0.001029,0.001023,0.000976,0.000291,0.000452,0.000293,0.084678,0.140778,0.000639,0.000000,0.000510,0.000339,0.000338,0.003778,0.000622,0.002742,0.001356,0.001356,0.001013,0.001030
AL-HB11,0.003721,0.003746,0.000738,0.000994,0.005270,0.001714,0.017843,0.143832,1.000000,0.009483,0.077000,0.150192,0.005191,0.005161,0.006106,0.000000,0.000745,0.000000,0.075092,0.108327,0.009795,0.000000,0.001339,0.000000,0.000000,0.001431,0.002269,0.004687,0.001246,0.001246,0.001033,0.002191


### ...and figure out what we want to use as our baseline

We'll use `IL-HB4066` because it seems to have some lines in common with other bills

In [66]:
similarity['AL-HB8'].sort_values(ascending=False).head(20)

bill_id
AL-HB8        1.000000
WV-SB618      0.299935
AL-HB11       0.150192
MS-HR87       0.057761
MS-HC62       0.057430
MS-SR56       0.055200
OK-HR1038     0.039869
ID-H0377      0.031492
MI-SB0460     0.028747
ID-H0375      0.020405
SC-H4325      0.020303
MO-HB952      0.015050
WA-SB5194     0.011008
UT-SR0901     0.007946
UT-HR0901     0.007892
WA-HB1477     0.004778
CA-AB473      0.004770
CO-HB1250     0.003440
CT-HB06666    0.003030
IL-HB4066     0.002649
Name: AL-HB8, dtype: float64

In [67]:
# Find the top 50 most similar, sorted by similarity
# and give me their bills numbers (they're the index)
# and then convert them into a list
most_similar = list(similarity['AL-HB8'].sort_values(ascending=False).head(15).index)
most_similar[:5]

['AL-HB8', 'WV-SB618', 'AL-HB11', 'MS-HR87', 'MS-HC62']

### Learn the phrases that are inside of IL-HB4066

These are the phrases we want to highlight - maybe we didn't use ngrams for similarity, but we're using ngrams down here for highlighting??? (for example)

In [68]:
bill_text = df.loc['AL-HB8'].bill_text
bill_text[:100]

'1 HB8\n\n2 214376-1\n\n3 By Representatives Pringle and Mooney\n\n4 RFD: Education Policy \n\n5 First Read: '

In [73]:
from sklearn.feature_extraction.text import CountVectorizer

# Find everything between 2-6 words
# and token_pattern means include things like "a" or "I" or "at"
vectorizer = CountVectorizer(ngram_range=(2,6), token_pattern='(?u)\\b\\w+\\b')

# Learn the words
vectorizer.fit([bill_text])
phrases = vectorizer.get_feature_names()
phrases[:10]

['0 1',
 '0 1 214376',
 '0 1 214376 1',
 '0 1 214376 1 n',
 '0 1 214376 1 n 06',
 '01 2021',
 '01 2021 ahp',
 '01 2021 ahp cmg',
 '01 2021 ahp cmg lsa2021',
 '01 2021 ahp cmg lsa2021 1417']

### Use the highlighter

In [74]:
# You can always cut and paste this code as long as you have 'phrases'

from rich.console import Console
from rich.highlighter import RegexHighlighter
from rich.theme import Theme

# You can run this code if you have a variable called
# "phrases" that is... the phrases you want to match.
class PhraseHighlighter(RegexHighlighter):
    base_style = "highlighter."
    highlights = [f"(?P<phrase>{phrase})" for phrase in phrases]

theme = Theme({"highlighter.phrase": "on pale_turquoise1"})
highlighter = Console(highlighter=PhraseHighlighter(), theme=theme)

In [75]:
for doc in most_similar:
    content = df.loc[doc].bill_text.lower() \
        .replace(".", "") \
        .replace(",", "") \
        .replace(";", "") \
        .replace('"', "") \
        .replace('“', "") \
        .replace('”', "") \
        .replace(")", "") \
        .replace("(", "")

    highlighter.print(f"[bold white on purple]{doc}[/bold white on purple]")
    highlighter.print(content)
    print("\n\n\n\n--------\n\n\n\n")

AL-HB8

1 hb8

2 214376-1

3 by representatives pringle and mooney

4 rfd: education policy 

5 first read: 11-jan-22 

6 pfd: 06/28/2021

 

page 0



1 214376-1:n:06/01/2021:ahp*/cmg lsa2021-1417

2  

3  

4  

5  

6  

7  

8 synopsis:         this bill would prohibit public k-12 schools

9 and public institutions of higher education from

10 teaching certain concepts regarding race or sex

11 such as critical race theory

12 this bill would prohibit a k-12 school or

13 public institution of higher education from

14 imposing any penalty upon or discriminating against

15 a student on account of his or her refusal to

16 support believe endorse embrace confess act

17 upon or otherwise assent to divisive concepts

18 such as critical race theory

19 this bill would also authorize a public k-12

20 school or public institution of higher education to

21 promote racial cultural or ethnic diversity or

22 inclusiveness provided the efforts of the school

23 or institution are consistent with the requirements

24 of this bill

25  

26 a bill

27 to be entitled

page 1



1 an act

2  

3 relating to education to prohibit public k-12

4 schools and public institutions of higher education from

5 teaching certain concepts regarding race or sex

6 be it enacted by the legislature of alabama:

7 section 1 for the purposes of this act the

8 following terms have the following meanings:

9 1 a divisive concept any of the following

10 concepts:

11 1 that one race or sex is inherently superior to

12 another race or sex

13 2 that this state or the united states is

14 fundamentally racist or sexist

15 3 that an individual by virtue of his or her race

16 or sex is inherently racist sexist or oppressive whether

17 consciously or unconsciously

18 4 that an individual should be discriminated

19 against or receive adverse treatment solely or partly because

20 of his or her race or sex

21 5 that members of one race or sex cannot and should

22 not attempt to treat others without respect to race or sex

23 6 that an individual's moral character is

24 necessarily determined by his or her race or sex

25 7 that an individual by virtue of his or her race

26 or sex bears responsibility for actions committed in the past

27 by other members of the same race or sex

page 2



1 8 that any individual should feel discomfort

2 guilt anguish or any other form of psychological distress on

3 account of his or her race or sex

4 9 that meritocracy or traits such as a hard work

5 ethic are racist or sexist or were created by a particular

6 race to oppress another race

7 b this term includes any other form of race or sex

8 stereotyping or any other form of race or sex scapegoating

9 2 race or sex scapegoating assigning fault

10 blame or bias to a race or sex or to members of a race or

11 sex because of their race or sex the term includes any claim

12 that consciously or unconsciously and by virtue of his or

13 her race or sex members of any 





--------






WV-SB618

west virginia-2021-sb618-introduced





west virginia
legislature


2021 regular session


introduced


senate bill 618


by senator azinger


[introduced march 10 2021
referred

to the committee on education and then to the committee on the judiciary]


a bill to amend the code
of west virginia 1931 as amended by adding thereto a new section designated
§18-2-44 relating to the teaching of divisive acts and critical race
theory in west virginia schools providing definitions prohibiting public
schools and charter schools from teaching or training students to believe
certain divisive concepts allowing a teacher to be dismissed or not reemployed
for teaching or training students to believe certain divisive concepts
providing certain construction providing for promulgation of rules providing
for codification and providing an effective date


be it enacted by the
legislature of west virginia:











article 2 state board
of education


§18-2-44 curriculum promoting
divisive acts and critical race theory prohibited











a as used in this
section: 


1 divisive concepts
means the concepts that: 


a one race or sex is
inherently superior to another race or sex


b the united states is
fundamentally racist or sexist


c an individual by virtue
of his or her race or sex is inherently racist sexist or oppressive whether
consciously or unconsciously


d an individual should
be discriminated against or receive adverse treatment solely or partly because
of his or her race or sex


e members of one race
or sex cannot and should not attempt to treat others without respect to race or
sex


f an individual’s
moral character is necessarily determined by his or her race or sex


g an individual by
virtue of his or her race or sex bears responsibility for actions committed in
the past by other members of the same race or sex


h any individual
should feel discomfort guilt anguish or any other form of psychological
distress on account of his or her race or sex


i meritocracy or
traits such as a hard work ethic are racist or sexist or were created by a
particular race to oppress another race or


j any other form of
race or sex stereotyping or any other form of race or sex scapegoating


2 race or sex
stereotyping means ascribing character traits values moral and ethical
codes privileges status or beliefs to a race or sex or an individual because
of his or her race or sex and 


3 race or sex
scapegoating means assigning fault blame or bias to a race or sex or to
members of a race or sex because of their race or sex it also includes any
claim that consciously or unconsciously and by virtue of his or her race or
sex members of any race are inherently racist or are inherently inclined to
oppress others or that members of a sex are inherently sexist or inclined to
oppress others 


b public school
districts and public charter schools in this state shall be prohibited from
teaching instructing or training any student to believe any of the divisive
concepts set forth in subsection a of this section 


c a teacher may be
dismissed or not reemployed for teaching instructing or training any student
to believe any of the divisive concepts set forth in subsection a of this
section 


d nothing in this
section shall be construed to prohibit discussing as part of a larger course
of academic instruction the divisive concepts provided for in subsection a of
this section in an objective manner and without endorsement 


e the west virginia state
board of education may promulgate rules to implement and monitor compliance
with the provisions of this section 


f this act shall
become effective july 1 2021 


note: the purpose of this bill is
to prohibit the teaching of divisive acts and
critical race theory in west virginia public and charter schools


strike-throughs indicate language
that would be stricken from a heading or the present law and underscoring
indicates new language that would be added

...

enrolled copy sr 901

1 senate resolution on critical race theory in

2 public education

3 2021 first senate extraordinary session

4 state of utah

5 chief sponsor:  lincoln fillmore

6  

7 long title

8 general description:

9 this resolution makes recommendations to the utah state board of education

10 regarding critical race theory

11 highlighted provisions:

12 this resolution:

13 < recognizes the importance of appropriate education on history civil rights and

14 racism

15 < identifies risks of critical race theory in public education and

16 < makes recommendations to the utah state board of education regarding the

17 prohibition of certain concepts

18 special clauses:

19 none

20  

21 be it resolved by the senate of the state of utah:

22 whereas educating students in utah's public education system on history civil

23 rights racism and the negative impacts racism has had throughout history is necessary and

24 should be done in a thoughtful historically accurate and appropriate manner

25 whereas some concepts contained in critical race theory degrade important societal

26 values and if introduced in classrooms would harm students' learning in the public 
education

27 system

28 whereas the utah state board of education has the statutory responsibility to

29 establish rules and minimum standards for the public schools including curriculum and



sr 901 enrolled copy

- 2 -

30 instruction requirements:

31 now therefore be it resolved that the senate strongly recommends that

32 the utah state board of education review standards for curriculum and ensure that the

33 following concepts are not included in the curriculum standards:

34 < that one race is inherently superior or inferior to another race

35 < that an individual should be discriminated against or receive adverse treatment

36 because of the individual's race or

37 < that an individual's moral character is determined by the individual's race

38 be it further resolved that the senate strongly recommends that no training or

39 training material that the utah state board of education or a local education agency 
provides

40 include the concepts described in the preceding paragraph

41 be it further resolved that the utah state board of education report back to

42 the education interim committee regarding the board's progress related to this resolution





--------






UT-HR0901

enrolled copy hr 901

1 house resolution on critical race theory in

2 public education

3 2021 first house extraordinary session

4 state of utah

5 chief sponsor:  steve r christiansen

6  

7 long title

8 general description:

9 this resolution makes recommendations to the utah state board of education

10 regarding critical race theory

11 highlighted provisions:

12 this resolution:

13 < recognizes the importance of appropriate education on history civil rights and

14 racism

15 < identifies risks of critical race theory in public education and

16 < makes recommendations to the utah state board of education regarding the

17 prohibition of certain concepts

18 special clauses:

19 none

20  

21 be it resolved by the house of representatives of the state of utah:

22 whereas educating students in utah's public education system on history civil

23 rights racism and the negative impacts racism has had throughout history is necessary and

24 should be done in a thoughtful historically accurate and appropriate manner

25 whereas some concepts contained in critical race theory degrade important societal

26 values and if introduced in classrooms would harm students' learning in the public 
education

27 system

28 whereas the utah state board of education has the statutory responsibility to

29 establish rules and minimum standards for the public schools including curriculum and



hr 901 enrolled copy

- 2 -

30 instruction requirements:

31 now therefore be it resolved that the house of representatives strongly

32 recommends that the utah state board of education review standards for curriculum and

33 ensure that the following concepts are not included in the curriculum standards:

34 < that one race is inherently superior or inferior to another race

35 < that an individual should be discriminated against or receive adverse treatment

36 because of the individual's race or

37 < that an individual's moral character is determined by the individual's race

38 be it further resolved that the house of representatives strongly recommends

39 that no training or training material that the utah state board of education or a local 
education

40 agency provides include the concepts described in the preceding paragraph

41 be it further resolved that the utah state board of education report back to

42 the education interim committee regarding the board's progress related to this resolution





--------






In [79]:
df['contains_superior'] = df.bill_text.str.lower().str.contains("is inherently superior").astype(int)
df.head()

,state,bill_number,title,bill_text,contains_superior
bill_id,,,,,
UT-HR0901,UT,HR0901,House Resolution on Critical Race Theory in Pu...,Enrolled Copy H.R. 901\n\n1 HOUSE RESOLUTION O...,1
UT-SR0901,UT,SR0901,Senate Resolution on Critical Race Theory in P...,Enrolled Copy S.R. 901\n\n1 SENATE RESOLUTION ...,1
IL-HR0365,IL,HR0365,CRITICAL RACE THEORY GRANTS,Illinois-2021-HR0365-Introduced\n\n\n\n\t \n\n...,0
IL-HB4066,IL,HB4066,CRITICAL RACE THEORY ACADEMY,Illinois-2021-HB4066-Introduced\n\n\n\n\t \n\n...,0
OK-HR1038,OK,HR1038,Resolution; discouraging schools from mandatin...,Resolution \n\nENROLLED HOUSE \n\nRESOLUTION N...,1


In [80]:
pd.crosstab(df.contains_superior, df.state)

state,AL,CA,CO,CT,DC,HI,ID,IL,LA,MA,MI,MN,MO,MS,OK,SC,UT,WA,WV
contains_superior,,,,,,,,,,,,,,,,,,,
0,0,2,1,1,1,4,0,2,1,1,0,3,1,0,0,0,0,2,1
1,2,0,0,0,0,0,2,0,0,0,1,0,0,3,1,1,2,0,0
